In [1]:
import tensorflow as tf
import time

In [9]:
class FileDataset(tf.data.Dataset):
    def read_files_in_batches(num_samples):
        #open file
        time.sleep(0.03)
        for sample_idx in range(num_samples):
            time.sleep(0.015)
            yield (sample_idx,)
    def __new__(cls, num_samples = 3):
        return tf.data.Dataset.from_generator(
            cls.read_files_in_batches,
            output_signature = tf.TensorSpec(shape = (1,), dtype = tf.int64),
            args = (num_samples,)
        )

In [11]:
def benchmark(dataset, num_epochs = 2):
    for epoch in range(num_epochs):
        for sample in dataset:
            time.sleep(0.01)

In [17]:
%%timeit
benchmark(FileDataset())

255 ms ± 3.47 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [25]:
%%timeit
benchmark(FileDataset().prefetch(tf.data.AUTOTUNE))

256 ms ± 6 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [29]:
dataset = tf.data.Dataset.range(5)
for d in dataset:
    print(d.numpy())

0
1
2
3
4


In [31]:
dataset = dataset.map(lambda x: x**2)
for d in dataset:
    print(d.numpy())

0
1
4
9
16


In [33]:
dataset = dataset.cache()
list(dataset.as_numpy_iterator())

[0, 1, 4, 9, 16]

In [41]:
def mapped_func(s):
    tf.py_function(lambda: time.sleep(0.03), [], ())
    return s

In [45]:
FileDataset().map(mapped_func)

<_MapDataset element_spec=TensorSpec(shape=(1,), dtype=tf.int64, name=None)>

In [61]:
%%timeit -n1 -r1
benchmark(FileDataset().map(mapped_func), 10)

2.13 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [83]:
%%timeit -n1 -r1
benchmark(FileDataset().map(mapped_func).cache(), 10)

595 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)
